## Writeup

### Author: Pengmian Yan

---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]:  ../after_calibration/calibration1-undistortion.jpg "Undistorted"
[image2]: ../output_images/test1_undist.jpg "straight_lines1_undist"
[image3]: ../output_images/straight_lines1_color_binary.jpg "Color Binary Example"
[image4]: ../output_images/straight_lines1_binary.jpg "gray Binary Example"
[image5]: ../output_images/straight_lines2_binary_and_warped.jpg "Warp Example"
[image6]: ../output_images/test1_warped_lines.png "Fit Visual"
[image7]: ../output_images/test1_with_lane.jpg "Output"
[video1]: ../project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Advanced-Lane-Lines/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

You're reading it!

### Camera Calibration

#### 1. Computation of the camera matrix and distortion coefficients and an example of a distortion corrected calibration image.

The code for this step is contained in the second code cell of the IPython notebook located in "../Result/Pipeline_for_advanced_lane_line_detection_in_singel_images.ipynb" below the Camera Calibration or "../Result/Camera_Calibration_P2_PengmianYan.ipynb".  

I start by preparing "object points", which will be the (x, y, z) coordinates of the 9x6 chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

![alt text][image1]

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:

![alt text][image2]

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

To find the related points on lane lines, I created a thresholded binary image with a combination of color and gradient thresholds in the third code cell. As an example I used straight_lines1_undist.jpg as input. To find out the optimal thresholds I created the colorful binary image. On that is clearly to see which parts of the lane lines were detected by the X-gradient threshold (green) and which parts were detected by the color threshold (red) by stacking the channels and seeing the individual components.

![alt text][image3]

Through the colorful binary image I found the best thresholds:


| SobelX        |   s_channel from hls   |   l_channel from luv   |   b_channel from lab
|:----------------:|:------------------------:| :----------------------: | :-------------: |
|    140, 255    |      50, 170        |      225, 255        |     155, 200

So the output image of this step is the fusion of red part and the green part. The biary imag of straight_lines1 is:

![alt text][image4]

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `warper()`, which appears in lines 1 through 8 in the file `example.py` (output_images/examples/example.py) (or, for example, in the 3rd code cell of the IPython notebook).  The `warper()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
src = np.float32(
    [[600, 446],
    [237, img_size[1]],
    [1082, img_size[1]],
    [680, 446]])

dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])
```

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 600, 446      | 320, 0        | 
| 237, 720      | 320, 720      |
| 1082, 720     | 960, 720      |
| 680, 446      | 960, 0        |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.Following are the binary image and warped bianry image of straight_lines2.jpg.

![alt text][image5]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Then I used function "find_lane_pixels(binary_warped)"(It's defined in the 5th code cell in Pipeline_for_advanced_lane_line_detection_in_singel_images.ipynb.) to find the pixels related to the lane on the warped bianry image. The principle is to make a histogram of the bottom half of image to get a basepoint and from there to search the pixels in next slice window. 

Then I used function "fit_polynomial(binary_warped)"(It's defined in the 6th code cell in Pipeline_for_advanced_lane_line_detection_in_singel_images.ipynb.) to fit my lane lines with a 2nd order polynomial kinda like this:

![alt text][image6]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I defined a function calls "measure_curvature_real(ploty, left_fit_cr, right_fit_cr)" to compute the curvature in the 7th code cell of Pipeline_for_advanced_lane_line_detection_in_singel_images.ipynb. This function returns the curvatur of the left and right lane lines, which seems to be a little bit different. The mean value of these two curvatur would be displayed on the images. 

The funciton "estimate_lateral_Positon(ploty, left_fit, right_fit, xmax)" was defined in the 8th code cell of Pipeline_for_advanced_lane_line_detection_in_singel_images.ipynb. For the position of the vehicle, its assumed that the camera is mounted at the center of the car and the deviation of the midpoint of the lane from the center of the image is the lateral Position offset. After that it converted to the real world values in meter.

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step direktly in the main function, which is the last code cell in Pipeline_for_advanced_lane_line_detection_in_singel_images.ipynb.  The output images are stored in ../output_images. Here is an example of my result on test1.jpg:

![alt text][image7]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's my improved video [link to my video result](./project_video_output2.mp4)

I created global objects line_left and line_right through class Line(). If the detected lines in some frame can not pass the sanity check, the lines in last frame will be used temporally. 

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Using the combination of color and gradient thresholds makes the longitudinal lines like lane lins better detected. And the perspective transform makes it easier to fucus on the lines of ego-lane and possible to compute the curvature radius. 

If the lane lines go through dark shadows or coverd by other cars, it would make my pipeline likely to fail. I would improve the stability and the smooty of my pipleline with the average of severy lines in the past. It would also be a good idea to use the  low-pass filter to the lane soother. 

Improved Pipelins are:

Pipeline_for_advanced_lane_line_detection_in_singel_images_advanced.ipynb

Pipeline_for_advanced_lane_line_detection_in_videos_advanced.ipynb
